In [ ]:
# default_exp annotation.yolo_adapter

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import csv
import sys
import argparse
import logging
from os.path import join, splitext, basename
from mlcore.io.core import scan_files
from mlcore.image.pillow_tools import get_image_size
from mlcore.annotation.core import Annotation, Region, RegionShape, create_annotation_id

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

FIELD_NAMES = ['class_number', 'c_x', 'c_y', 'width', 'height']

In [ ]:
# export

logger = logging.getLogger(__name__)

# YOLO Annotation Adapter
> YOLO annotation adapter. Converts the bounding boxes into YOLO format.

In [ ]:
# export


def read_annotations(annotations_folder, files_source, categories):
    """
    Reads YOLO annotations from the annotations folder.
    `annotations_folder`: the path to the folder containing the annotation files
    `files_source`: the path to the folder containing the source files
    `categories`: the categories used
    return: the annotations
    """

    annotations = {}
    annotation_files = scan_files(annotations_folder, file_extensions='.txt')
    categories_len = len(categories)
    for annotation_file in annotation_files:
        with open(annotation_file, newline='') as csvfile:
            annotation_file_name = basename(annotation_file)
            file_name, _ = splitext(annotation_file_name)
            file_path = join(files_source, '{}{}'.format(file_name, '.jpg'))
            annotation_id = create_annotation_id(file_path)
            if annotation_id is None:
                logger.warning('File not found, skip annotations at path: {}'.format(annotation_file))
                continue
            if annotation_id not in annotations:
                annotations[annotation_id] = Annotation(annotation_id=annotation_id, file_path=file_path)

            reader = csv.DictReader(csvfile, fieldnames=FIELD_NAMES, delimiter=' ')
            _, image_width, image_height = get_image_size(file_path)
            for row in reader:
                c_x = float(row["c_x"])
                c_y = float(row["c_y"])
                width = float(row["width"])
                height = float(row["height"])
                class_number = int(row["class_number"])
                x_min = _denormalize_value(c_x - (width / 2), image_width)
                y_min = _denormalize_value(c_y - (height / 2), image_height)
                x_max = x_min + _denormalize_value(width, image_width)
                y_max = y_min + _denormalize_value(height, image_height)
                points_x = [x_min, x_max]
                points_y = [y_min, y_max]

                labels = [categories[class_number]] if class_number < categories_len else []
                region = Region(shape=RegionShape.RECTANGLE, points_x=points_x, points_y=points_y, labels=labels)
                annotations[annotation_id].regions.append(region)
    return annotations

In [ ]:
# export


def write_annotations(annotations_folder, annotations, categories):
    """
    Writes YOLO annotations to the annotations folder.
    `annotations_folder`: the path to the folder containing the annotation files
    `annotations`: the annotations to write
    `categories`: the categories used
    """
    for annotation in annotations.values():
        file_name, _ = splitext(basename(annotation.file_path))
        annotations_file = join(annotations_folder, '{}{}'.format(file_name, '.txt'))
        _, image_width, image_height = get_image_size(annotation.file_path)
        with open(annotations_file, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=FIELD_NAMES, delimiter=' ')

            for index, region in enumerate(annotation.regions):
                if region.shape != RegionShape.RECTANGLE:
                    logger.warning('Unsupported shape {}, skip region {} at path: {}'.format(region.shape, index,
                                                                                             annotations_file))
                    continue

                x_min, x_max = region.points_x
                y_min, y_max = region.points_y
                width = x_max - x_min
                height = y_max - y_min
                c_x = _normalize_value(x_min + width / 2, image_width)
                c_y = _normalize_value(y_min + height / 2, image_height)
                width = _normalize_value(width, image_width)
                height = _normalize_value(height, image_height)
                label = region.labels[0] if len(region.labels) else ''
                class_number = categories.index(label)

                writer.writerow({'class_number': class_number,
                                 'c_x': c_x,
                                 'c_y': c_y,
                                 'width': width,
                                 'height': height})

In [ ]:
# export


def _denormalize_value(value, metric):
    """
    Denormalize a bouning box value
    `value`: the value to denormalize
    `metric`: the metric to denormalize from
    return: the denormalized value
    """
    return int(value * metric)

In [ ]:
# export


def _normalize_value(value, metric):
    """
    Normalize a bouning box value
    `value`: the value to normalize
    `metric`: the metric to normalize against
    return: the normalized value
    """
    return float(value) / metric

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.annotation.yolo_adapter [parameters]`

The following parameters are supported:
- `[annotations_folder]`: The path to the YOLO annotations folder. (e.g.: *imagesets/image_object_detection/car_damage/labels/trainval*)
- `--files_source`: The path to the folder containing the source files (e.g.: *imagesets/image_object_detection/car_damage/images/trainval*)
- `--categories`: The path to the categories file. (e.g.: *imagesets/image_object_detection/car_damage/categories.txt*)

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("annotations_folder",
                        help="The path to the annotations folder.")
    parser.add_argument("--files_source",
                        help="The path to the folder containing the source files.")
    parser.add_argument("--categories",
                        help="The path to the categories file.")
    args = parser.parse_args()

    read_annotations(args.annotations_folder, args.files_source, args.categories)
